In [ ]:
import numpy as np
import pandas as pd
import os
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

os.chdir(r'/kaggle/working')
os.listdir('/kaggle/input')

# Function to load the dataset
def create_dataset(path, num_fractured=12000, num_normal=12000):
    """
    Load X-ray dataset, selecting the first num_fractured fractured images and the first num_normal normal images.
    Label fractured as 1 and normal as 0.
    """
    dataset = []
    fractured_count = 0
    normal_count = 0

    for id_p in os.listdir(path):
        patient_id = id_p
        path_id = path + '/' + str(id_p)
        for lab in os.listdir(path_id):
            if lab.split('_')[-1] == 'positive':
                label = 1
            elif lab.split('_')[-1] == 'negative':
                label = 0
            path_l = path_id + '/' + str(lab)
            for img in os.listdir(path_l):
                img_path = os.path.join(path_l, img)
                if label == 1 and fractured_count < num_fractured:
                    fractured_count+=1
                    dataset.append({
                        'label': label,
                        'image_path': img_path
                    })
                if label == 0 and normal_count < num_normal:
                    normal_count+=1
                    dataset.append({
                        'label': label,
                        'image_path': img_path
                    })
    print(normal_count,fractured_count)
    random.shuffle(dataset)
    return dataset

# Load dataset into train and test df
train_dir = ('/kaggle/input/better-mura/MURA-v1.1/MURA-v1.1/train_augmented/XR_FINGER')
traindata = create_dataset(train_dir)
trainfilepaths = []
trainlabels = []
output_directory = '/kaggle/working'

for row in traindata:
    trainfilepaths.append(row['image_path'])
    trainlabels.append(row['label'])

trainfilepaths = pd.Series(trainfilepaths, name='Filepath').astype(str)
trainlabels = pd.Series(trainlabels, name='Label').astype(str)
train_df = pd.concat([trainfilepaths, trainlabels], axis=1)

test_dir = ('/kaggle/input/better-mura/MURA-v1.1/MURA-v1.1/valid/XR_FINGER')
testdata = create_dataset(test_dir)
testfilepaths = []
testlabels = []

for row in testdata:
    testlabels.append(row['label'])
    testfilepaths.append(row['image_path'])

testfilepaths = pd.Series(testfilepaths, name='Filepath').astype(str)
testlabels = pd.Series(testlabels, name='Label').astype(str)
test_df = pd.concat([testfilepaths, testlabels], axis=1)

#ImageDataGenerator for data augmentation and preprocessing
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input,
    validation_split=0.2)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input)
# Data generators
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),  # EfficientNetB0 expects 224x224 input size
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=False
)

In [ ]:
pretrained_model = tf.keras.applications.resnet50.ResNet50(input_shape=(224, 224, 3), include_top=False, weights='imagenet', pooling='avg')

# Freeze the layers of the pretrained ResNet50 model
pretrained_model.trainable = True

# Add dense layers with dropout for regularization
inputs = pretrained_model.input
x = tf.keras.layers.Dense(256, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dropout(0.5)(x)  # Add dropout
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)  # Add dropout
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)  # Add dropout
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)


model = tf.keras.Model(inputs, outputs)

# Compile the model with Adam optimizer and binary cross-entropy loss
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Define checkpoint path with a placeholder for epoch and metric values
checkpoint_path = "/kaggle/working/FingerResNet50Callback.keras"

# Define early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)

# Define learning rate reduction callback
learn_control = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.2, min_lr=1e-7)

# Create a custom model checkpoint callback
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
                        filepath=checkpoint_path,
                        monitor='val_loss',
                        save_best_only=True,
                        save_weights_only=False,  # Save the whole model
                        mode='min',
                        verbose=1
                    )

# Combine all callbacks
callbacks = [early_stopping, learn_control, model_checkpoint]


# Train model
print("-------Training Finger" + "-------")
history = model.fit(train_images, validation_data=val_images, epochs=60, callbacks=callbacks)

# Evaluate model
results = model.evaluate(test_images, verbose=1)
print("Finger Results:")
print(results)
print(f"Test Accuracy: {np.round(results[1] * 100, 2)}%")
modelpath="/kaggle/working/FingerResnet50.keras"
model.save(modelpath)